In [40]:
# pip install regex
import re
from dataclasses import dataclass
from datetime import datetime

@dataclass
class Tide:
    t_date: int
    t_type: str # High or Low
    t_height: float
    t_time: datetime

tides = []


my_tide = Tide(1,"asdfadf",4.1,datetime.strptime("15:00",'%H:%M').time())
tides.append(my_tide)
my_tide = Tide(2,"xxsdfadf",5.3,datetime.strptime("14:22",'%H:%M').time())
print(my_tide)
tides.append(my_tide)
print("22")
print(tides)


with open("./hydro_2024.08.04.txt","r") as infile:
    lines = infile.readlines()



Tide(t_date=2, t_type='xxsdfadf', t_height=5.3, t_time=datetime.time(14, 22))
22
[Tide(t_date=1, t_type='asdfadf', t_height=4.1, t_time=datetime.time(15, 0)), Tide(t_date=2, t_type='xxsdfadf', t_height=5.3, t_time=datetime.time(14, 22))]


In [25]:
cleaned_lines = [line.strip() for line in lines]
high_lows = [line for line in cleaned_lines if line.startswith(("Low","High","Fri","Sat","Sun","Mon","Tue","Wed","Thur"))]

In [26]:
formatted_tides = []
for line in high_lows:
    words = line.split()
    print(words)
   
    # If the sentence starts with Sun, Mon, etc, then keep words 1 (base 0)
    # (date, but not day). Month can be inferred downstream
    if words[0] in ("Sun","Mon","Tues","Weds","Thurs","Fri","Sat"):
        formatted_tides.append("----------------")
        formatted_tides.append(f"{words[1]},")
        continue
    # After previous block, 7 or more words means a High or Low water sentence. 
    # Words 0 (base zero), 3 and 6 are respectively High/Low, tide-height, and 
    # time. Other words and characters are discarded
    if len(words) >= 7:
        formatted_tides.append(f"{words[0]},{words[6].rstrip('.')},{words[3]},")
        continue     
for line in formatted_tides:
    print(line)

['Sun', '04', 'AugNew', 'moon', 'on', 'this', 'day']
['Low', 'Water', 'of', '0.7', 'metres,', 'at', '01:08.']
['High', 'Water', 'of', '3.4', 'metres,', 'at', '07:46.']
['Low', 'Water', 'of', '0.7', 'metres,', 'at', '13:21.']
['High', 'Water', 'of', '3.7', 'metres,', 'at', '19:59.']
['Mon', '05', 'Aug']
['Low', 'Water', 'of', '0.6', 'metres,', 'at', '01:53.']
['High', 'Water', 'of', '3.5', 'metres,', 'at', '08:28.']
['Low', 'Water', 'of', '0.6', 'metres,', 'at', '14:03.']
['High', 'Water', 'of', '3.8', 'metres,', 'at', '20:37.']
['Tues', '06', 'Aug']
['Low', 'Water', 'of', '0.6', 'metres,', 'at', '02:33.']
['High', 'Water', 'of', '3.5', 'metres,', 'at', '09:01.']
['Low', 'Water', 'of', '0.6', 'metres,', 'at', '14:39.']
['High', 'Water', 'of', '3.8', 'metres,', 'at', '21:08.']
['Weds', '07', 'Aug']
['Low', 'Water', 'of', '0.6', 'metres,', 'at', '03:06.']
['High', 'Water', 'of', '3.5', 'metres,', 'at', '09:28.']
['Low', 'Water', 'of', '0.6', 'metres,', 'at', '15:10.']
['High', 'Water', 'o

In [27]:
lines2 = [line for line in formatted_tides if not line.startswith('-')]
print(lines2)

['04,', 'Low,01:08,0.7,', 'High,07:46,3.4,', 'Low,13:21,0.7,', 'High,19:59,3.7,', '05,', 'Low,01:53,0.6,', 'High,08:28,3.5,', 'Low,14:03,0.6,', 'High,20:37,3.8,', '06,', 'Low,02:33,0.6,', 'High,09:01,3.5,', 'Low,14:39,0.6,', 'High,21:08,3.8,', '07,', 'Low,03:06,0.6,', 'High,09:28,3.5,', 'Low,15:10,0.6,', 'High,21:33,3.7,', '08,', 'Low,03:33,0.6,', 'High,09:49,3.5,', 'Low,15:36,0.6,', 'High,21:55,3.7,', '09,', 'Low,03:56,0.6,', 'High,10:09,3.4,', 'Low,16:01,0.6,', 'High,22:18,3.6,', '10,', 'Low,04:19,0.6,', 'High,10:34,3.3,', 'Low,16:26,0.7,', 'High,22:45,3.4,']


In [28]:
tide_days = []
data = lines2
day_tides = []
current_day_tide = ""

# Iterate through the data
for item in data:
    # Check if the item has 2 characters (indicating the start of a record)
    if len(item) == 3:
        # If we have a current record, add it to the grouped_records list
        if current_day_tide:
            day_tides.append(current_day_tide)
        # Start a new record
        current_day_tide = item
    else:
        # Concatenate the item to the current record
        current_day_tide += item

# Add the last record (if any) to the grouped_records list
if current_day_tide:
    day_tides.append(current_day_tide)

# Print the grouped records
for tide in day_tides:
    print(tide)

04,Low,01:08,0.7,High,07:46,3.4,Low,13:21,0.7,High,19:59,3.7,
05,Low,01:53,0.6,High,08:28,3.5,Low,14:03,0.6,High,20:37,3.8,
06,Low,02:33,0.6,High,09:01,3.5,Low,14:39,0.6,High,21:08,3.8,
07,Low,03:06,0.6,High,09:28,3.5,Low,15:10,0.6,High,21:33,3.7,
08,Low,03:33,0.6,High,09:49,3.5,Low,15:36,0.6,High,21:55,3.7,
09,Low,03:56,0.6,High,10:09,3.4,Low,16:01,0.6,High,22:18,3.6,
10,Low,04:19,0.6,High,10:34,3.3,Low,16:26,0.7,High,22:45,3.4,


In [29]:
def insert_tidal_range(record):
    # Split the record by commas
    parts = record.split(',')
    
    # Extract the decimal values (skipping the first word)
    values = [float(part) for part in parts[1:] if part.replace('.', '', 1).isdigit()]
    
    # Calculate the difference between the highest and lowest values
    difference = max(values) - min(values)
    
    # Insert the difference between "Low" and "01:53"
    parts.insert(1, f"{difference:.1f}")
    
    # Join the modified parts back into a string
    modified_record = ','.join(parts)
    
    return modified_record

# Example usage:
#record = "05,Low,01:53,0.6,High,08:28,3.5,Low,14:03,0.6,High,20:37,3.8"
#result = insert_tidal_range(record)
#print(result)

In [30]:
# insert the tidal range - this is the difference between the max and
# min tidal heights for the day
tides_with_range = [insert_tidal_range(line) for line in day_tides]
for tide in tides_with_range:
    print(tide)

04,3.0,Low,01:08,0.7,High,07:46,3.4,Low,13:21,0.7,High,19:59,3.7,
05,3.2,Low,01:53,0.6,High,08:28,3.5,Low,14:03,0.6,High,20:37,3.8,
06,3.2,Low,02:33,0.6,High,09:01,3.5,Low,14:39,0.6,High,21:08,3.8,
07,3.1,Low,03:06,0.6,High,09:28,3.5,Low,15:10,0.6,High,21:33,3.7,
08,3.1,Low,03:33,0.6,High,09:49,3.5,Low,15:36,0.6,High,21:55,3.7,
09,3.0,Low,03:56,0.6,High,10:09,3.4,Low,16:01,0.6,High,22:18,3.6,
10,2.8,Low,04:19,0.6,High,10:34,3.3,Low,16:26,0.7,High,22:45,3.4,


In [31]:
# return the usable tides for the day. Assume there are always 2,
# from a set of 4, and sometimes 3. 
# For the 4-set case, I keep the middle two tides.
# More work needed for the 3-set case, as needs more judgment about
# which end-tide to keep.


def get_usable_tides_for_day(record):
     # Split the record by commas
    parts = record.split(',')
    # remove the last tide of the day, as typically, this is not
    # usable:
    parts = parts[:-4] 
    # for a 4-tide day, remove the first tide
    if len(parts) == 11:
        parts = parts[:2] + parts[5:]
    modified_record = ','.join(parts)
    return modified_record

In [32]:
usable_tides = [get_usable_tides_for_day(line) for line in tides_with_range]
for tide in usable_tides:
    print(tide)

04,3.0,High,07:46,3.4,Low,13:21,0.7
05,3.2,High,08:28,3.5,Low,14:03,0.6
06,3.2,High,09:01,3.5,Low,14:39,0.6
07,3.1,High,09:28,3.5,Low,15:10,0.6
08,3.1,High,09:49,3.5,Low,15:36,0.6
09,3.0,High,10:09,3.4,Low,16:01,0.6
10,2.8,High,10:34,3.3,Low,16:26,0.7


In [33]:
def set_month_year(record, month_year):
    parts = record.split(',')
    # word 0 is the date - add the month and year
    parts[0] = f"{parts[0]}/{month_year}"
    modified_record = ','.join(parts)
    return modified_record
    

In [34]:
full_tides = [set_month_year(line,"08/2024") for line in usable_tides]
for tide in full_tides:
    print(tide)

04/08/2024,3.0,High,07:46,3.4,Low,13:21,0.7
05/08/2024,3.2,High,08:28,3.5,Low,14:03,0.6
06/08/2024,3.2,High,09:01,3.5,Low,14:39,0.6
07/08/2024,3.1,High,09:28,3.5,Low,15:10,0.6
08/08/2024,3.1,High,09:49,3.5,Low,15:36,0.6
09/08/2024,3.0,High,10:09,3.4,Low,16:01,0.6
10/08/2024,2.8,High,10:34,3.3,Low,16:26,0.7
